In [1]:
import numpy as np
import pandas as pd
import sklearn as sk
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, mean_squared_error, r2_score
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor,RandomForestClassifier
from sklearn.neighbors import KNeighborsRegressor


## Data traitment

Collecte et traitement des données

In [2]:
# Collecte des données
X= pd.read_csv(r"X_train_filled.csv",index_col=0)
Y = pd.read_csv(r"C:\Users\collo\Documents\Etudes\Dauphine\Machine Learning\Projet\DAUPHINE_Machine_Learning\y_train.csv",index_col=0)
data_train = pd.merge(X, Y, left_index=True, right_index=True).sort_index()
data_train

,load_forecast,coal_power_available,gas_power_available,nucelear_power_available,wind_power_forecasts_average,solar_power_forecasts_average,wind_power_forecasts_std,solar_power_forecasts_std,predicted_spot_price,spot_id_delta
DELIVERY_START,,,,,,,,,,
2022-01-01 02:00:00+01:00,49439.0,3386.0,11487.0,44118.0,3035.0,0.0,79.248348,0.000000,116.808914,-36.874770
2022-01-01 03:00:00+01:00,46511.0,3386.0,11487.0,44118.0,3143.0,0.0,61.776532,0.000000,116.345711,-12.643588
2022-01-01 04:00:00+01:00,45158.0,3386.0,11487.0,44118.0,3288.0,0.0,44.291112,0.000000,108.858177,-1.950193
2022-01-01 05:00:00+01:00,44779.0,3386.0,11487.0,44118.0,3447.0,0.0,36.127588,0.000000,112.760529,1.938272
2022-01-01 06:00:00+01:00,45284.0,3386.0,11487.0,44118.0,3679.0,0.0,30.983023,0.000000,114.032677,0.199907
...,...,...,...,...,...,...,...,...,...,...
2023-03-29 19:00:00+02:00,50814.0,3386.0,11952.0,38320.0,7552.0,651.0,247.408490,7.821622,108.110000,6.029303
2023-03-29 20:00:00+02:00,50628.0,3386.0,11952.0,38320.0,8338.0,109.0,155.795012,2.534054,125.660000,13.576177
2023-03-29 21:00:00+02:00,48201.0,3386.0,11952.0,38320.0,9115.0,0.0,126.884684,0.000000,138.010000,17.478945


In [3]:
# Traitement
## Missing Data
data = data_train.dropna(axis=1, thresh=len(data_train)*0.25)
data = data.dropna()
#data = data.ffill()
X = data[[col for col in data.columns if col !='spot_id_delta']]
X_lag = X.shift(1)
#X_lag_1 = X.shift(-1)
X_lag.columns=[col+"_lag" for col in X_lag.columns]
#X_lag_1.columns=[col+"_lag1" for col in X_lag_1.columns]
X = pd.concat([X,X_lag],axis=1).bfill().ffill()

In [4]:
Y = data['spot_id_delta'].to_numpy()
## Format
X = pd.get_dummies(X)
## Split
X_train, X_test, y_train, y_test = sk.model_selection.train_test_split(X, Y, test_size=0.2, random_state=42)

## Model Selection

In [5]:
scalers = [None,StandardScaler(), MinMaxScaler(), RobustScaler()]
models = {
    'XGBoost' : XGBRegressor(),
    #'Linear Regression': LinearRegression(),
    #'Ridge Regression': Ridge(),
    #'Lasso Regression': Lasso(),
    #'ElasticNet': ElasticNet(),
    #'Support Vector Regression': SVR(),
    #'Decision Tree Regression': DecisionTreeRegressor(),
    'Random Forest Regression': RandomForestRegressor(),
    'Gradient Boosting Regression': GradientBoostingRegressor(),
    #'K-Neighbors Regression': KNeighborsRegressor()
    }

res = list()
for mod in models.values() :
    print(f"Model: {mod.__class__.__name__}")
    for scaler in scalers:
        # Appliquer le scaler
        if scaler is None :
            X_train_scaled = X_train
            X_test_scaled = X_test
        else :
            X_train_scaled = scaler.fit_transform(X_train)
            X_test_scaled = scaler.transform(X_test)
            
        # Initialiser le modèle
        np.random.RandomState(42)
        model = mod
        
        # Entraîner le modèle de base
        model.fit(X_train_scaled, y_train)
        
        # Évaluation du modèle
        y_pred = model.predict(X_test_scaled)
        mse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        # Validation croisée
        cv_scores = sk.model_selection.cross_val_score(model, X_train_scaled, y_train, cv=10, scoring='neg_mean_squared_error')
        res.append([mod.__class__.__name__,scaler.__class__.__name__,mse,r2,cv_scores])


Model: XGBRegressor
Model: LinearRegression
Model: Ridge
Model: SVR
Model: DecisionTreeRegressor
Model: RandomForestRegressor
Model: GradientBoostingRegressor


In [6]:
res = pd.DataFrame(res,columns=["model","scaler","mse","r2","cv_score"]).sort_values(by="r2",ascending=False)
res["cv_mean"] = res['cv_score'].apply(lambda x : np.mean(x))
res

,model,scaler,mse,r2,cv_score,cv_mean
22,RandomForestRegressor,MinMaxScaler,382.953239,0.542016,"[-3200.4757827826966, -350.1627715836506, -290...",-1007.047296
23,RandomForestRegressor,RobustScaler,384.615573,0.540028,"[-2996.8853902093797, -342.6898389853039, -285...",-949.317443
21,RandomForestRegressor,StandardScaler,385.834215,0.538571,"[-3061.9821458178117, -353.9957070954516, -293...",-973.414050
20,RandomForestRegressor,NoneType,392.600707,0.530478,"[-2980.0516958705184, -354.3001736809046, -292...",-969.912411
0,XGBRegressor,NoneType,393.117643,0.529860,"[-1379.319516345745, -399.8278666758841, -359....",-837.921651
1,XGBRegressor,StandardScaler,393.117643,0.529860,"[-1379.319516345745, -399.8278666758841, -359....",-837.925615
2,XGBRegressor,MinMaxScaler,393.117643,0.529860,"[-1379.319516345745, -399.8278666758841, -359....",-837.925615
3,XGBRegressor,RobustScaler,393.117643,0.529860,"[-1379.319516345745, -399.8278666758841, -359....",-837.925615
16,DecisionTreeRegressor,NoneType,682.364484,0.183942,"[-3740.863430437411, -597.8697847591244, -784....",-1406.841845
18,DecisionTreeRegressor,MinMaxScaler,707.435064,0.153960,"[-3719.941048724071, -605.5282673340007, -798....",-1421.303765


## Model

Optimisation des paramètres

Creation et Validation du modèle

In [7]:
model = RandomForestRegressor(random_state=42) #**best_params)

# Entraîner le modèle sur les données d'entraînement complètes
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# Calcul des métriques
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"Mean Squared Error: {mse}")
print(f"R2 Score: {r2}")

Mean Squared Error: 381.66254771646743
R2 Score: 0.5435595749883542


In [8]:
model.fit(X_test,y_test)

RandomForestRegressor(random_state=42)

On utilie le model sur les valeurs manquantes

In [17]:
X_test = pd.read_csv(r"X_test_filled.csv",index_col=0)
X_test

,load_forecast,coal_power_available,gas_power_available,nucelear_power_available,wind_power_forecasts_average,solar_power_forecasts_average,wind_power_forecasts_std,solar_power_forecasts_std,predicted_spot_price
DELIVERY_START,,,,,,,,,
2023-04-02 00:00:00+02:00,45814.0,3386.0,10902.0,36705.0,6359.0,0.0,56.764535,0.0,111.145187
2023-04-02 01:00:00+02:00,44084.0,3386.0,10902.0,36705.0,6469.0,0.0,54.262133,0.0,110.423241
2023-04-02 02:00:00+02:00,43281.0,3386.0,10902.0,36705.0,6511.0,0.0,78.105928,0.0,110.569878
2023-04-02 03:00:00+02:00,40825.0,3386.0,10902.0,36705.0,6628.0,0.0,78.187557,0.0,82.747284
2023-04-02 04:00:00+02:00,39181.0,3386.0,10902.0,36705.0,6700.0,0.0,96.765484,0.0,82.842125
...,...,...,...,...,...,...,...,...,...
2023-10-24 19:00:00+02:00,49686.0,2226.0,11749.0,42980.0,4901.0,0.0,247.887323,0.0,125.670000
2023-10-24 20:00:00+02:00,53397.0,2226.0,11749.0,42980.0,5584.0,0.0,343.192642,0.0,139.580000
2023-10-24 21:00:00+02:00,50586.0,2226.0,11749.0,42980.0,6306.0,0.0,471.875973,0.0,147.930000


In [18]:
X_test_lag = X_test.shift(1)
#X_test_lag_1 = X_test.shift(-1)

X_test_lag.columns=[col+"_lag" for col in X_test_lag.columns]
#X_test_lag_1.columns=[col+"_lag1" for col in X_test_lag_1.columns]

X_test = pd.concat([X_test,X_test_lag],axis=1).bfill().ffill()
X_test

,load_forecast,coal_power_available,gas_power_available,nucelear_power_available,wind_power_forecasts_average,solar_power_forecasts_average,wind_power_forecasts_std,solar_power_forecasts_std,predicted_spot_price,load_forecast_lag,...,predicted_spot_price_lag,load_forecast_lag1,coal_power_available_lag1,gas_power_available_lag1,nucelear_power_available_lag1,wind_power_forecasts_average_lag1,solar_power_forecasts_average_lag1,wind_power_forecasts_std_lag1,solar_power_forecasts_std_lag1,predicted_spot_price_lag1
DELIVERY_START,,,,,,,,,,,,,,,,,,,,,
2023-04-02 00:00:00+02:00,45814.0,3386.0,10902.0,36705.0,6359.0,0.0,56.764535,0.0,111.145187,45814.0,...,111.145187,44084.0,3386.0,10902.0,36705.0,6469.0,0.0,54.262133,0.0,110.423241
2023-04-02 01:00:00+02:00,44084.0,3386.0,10902.0,36705.0,6469.0,0.0,54.262133,0.0,110.423241,45814.0,...,111.145187,43281.0,3386.0,10902.0,36705.0,6511.0,0.0,78.105928,0.0,110.569878
2023-04-02 02:00:00+02:00,43281.0,3386.0,10902.0,36705.0,6511.0,0.0,78.105928,0.0,110.569878,44084.0,...,110.423241,40825.0,3386.0,10902.0,36705.0,6628.0,0.0,78.187557,0.0,82.747284
2023-04-02 03:00:00+02:00,40825.0,3386.0,10902.0,36705.0,6628.0,0.0,78.187557,0.0,82.747284,43281.0,...,110.569878,39181.0,3386.0,10902.0,36705.0,6700.0,0.0,96.765484,0.0,82.842125
2023-04-02 04:00:00+02:00,39181.0,3386.0,10902.0,36705.0,6700.0,0.0,96.765484,0.0,82.842125,40825.0,...,82.747284,38928.0,3386.0,10902.0,36705.0,6750.0,0.0,133.267741,0.0,75.495132
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-24 19:00:00+02:00,49686.0,2226.0,11749.0,42980.0,4901.0,0.0,247.887323,0.0,125.670000,47307.0,...,99.820000,53397.0,2226.0,11749.0,42980.0,5584.0,0.0,343.192642,0.0,139.580000
2023-10-24 20:00:00+02:00,53397.0,2226.0,11749.0,42980.0,5584.0,0.0,343.192642,0.0,139.580000,49686.0,...,125.670000,50586.0,2226.0,11749.0,42980.0,6306.0,0.0,471.875973,0.0,147.930000
2023-10-24 21:00:00+02:00,50586.0,2226.0,11749.0,42980.0,6306.0,0.0,471.875973,0.0,147.930000,53397.0,...,139.580000,46777.0,2226.0,11749.0,42980.0,6959.0,0.0,595.528100,0.0,122.200000


In [19]:
X_test = pd.get_dummies(X_test)
y_test = model.predict(X_test)

In [20]:
Y_test = pd.Series(y_test,index = X_test.index)
Y_test.name = "spot_id_delta"
Y_test

DELIVERY_START
2023-04-02 00:00:00+02:00    1.250446
2023-04-02 01:00:00+02:00    1.706805
2023-04-02 02:00:00+02:00    6.213948
2023-04-02 03:00:00+02:00    1.785577
2023-04-02 04:00:00+02:00   -6.303174
                               ...   
2023-10-24 19:00:00+02:00   -1.395987
2023-10-24 20:00:00+02:00   -8.461613
2023-10-24 21:00:00+02:00   -7.715278
2023-10-24 22:00:00+02:00    0.153154
2023-10-24 23:00:00+02:00   -0.252774
Name: spot_id_delta, Length: 4942, dtype: float64

In [21]:
Y_test.to_csv("y_test.csv")